##### DNN CNN
    - 독립적인 정보
    - 입력(x) 간의 순서나 연관성을 고려하지 않는다
##### 시계열 : 시간의 연속적인 흐름
    - 시계열 데이터 : 날씨, 주식, 문장 --> 순서가 중요한 데이터
##### RNN 
    - 순환하는 구조
        - 시점1 (월요일) : 맑음(x1)  정보가 RNN에 들어온다 -> RNN 날씨가 맑았음(h1) 이라는 요약본을 생성
        - 시점2 (화요일) : 흐림(x2)  ->RNN 새로운정보(흐림, x2) + 어제의기억(맑았음, h1) 함께 고려
            어제 맑았는데 오늘 흐림 h2 이라는 새로운 요약본을 생성
        - 시점3 (수요일) : 비(x3) ->RNN 새로운정보(비, x3) 정보와 + 어제의기억(어제 맑았는데 오늘 흐림, h2) 새로운 상태 h3
        - 반복 
    - 알고리즘
        - 각 시점(time step)에서  1. 현재의 입력 과 2. 과거의 기억(hidden state ht-1) 받아서 3. 현재의 결과물과 4. 다음 시점으로 넘겨줄 최신 기억 ht 을 생성
        - ht 기억이 시계열 데이터의 맥락(Context) 저장하는 역할
    - 장점
        - 순서가 있는 데이터의 맥락을 학습
    - 한계
        - 기억력이 생각보다 짧다
        - 시계열 데이터가 길어지면(예 100단계 전의 정보)
        - 이전정보가 소실되거나 반대로 너무 강해져서 폭주가 되서 제대로 학습이 안된다
        - 장기 기억 의존성 문제 (Long-Term Dependency Problem)
##### LSTM & GRU
    - LSTM(Long Short Term Memory) : RNN 내부에 게이트(Gate) 복잡한 장치 -> 잊고, 기억할 정보를 관리
    - GRU(Gated Recurrent Unit) : LSTM 구조를 좀 더 단순화시킨 모델, LSTM성능은 비슷, 속도는 빠르다
##### RNN 핵심수식
    - 은닉상태계산
        - h1 = tanh(wht-1 + wxt - bh)
    - 출력계산